In [1]:
!pip install transformers evaluate transformers[torch]
!pip install py7zr #need to install for samsum dataset
!pip install -U datasets
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


### Load In Model & Dataset

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("ingeniumacademy/bart-cnn-samsum-finetuned")
model = AutoModelForSeq2SeqLM.from_pretrained("ingeniumacademy/bart-cnn-samsum-finetuned")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [15]:
from datasets import load_dataset

dataset = load_dataset("ingeniumacademy/samsum")
dataset

README.md:   0%|          | 0.00/685 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/923k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/966k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'sentences', 'sentence_id', 'dialog_id'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'sentences', 'sentence_id', 'dialog_id'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'sentences', 'sentence_id', 'dialog_id'],
        num_rows: 819
    })
})

### Prepare Dataset

In [4]:
def tokenize_inputs(example):
    start_prompt = "Summarize the following conversation.\n\n"
    end_prompt = "\n\nSummary: "
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    tokenized_prompt = tokenizer(prompt, padding='max_length', truncation=True, return_tensors='pt', max_length=512)
    tokenized_summary = tokenizer(example['summary'], padding='max_length', truncation=True, return_tensors='pt', max_length=512)

    example['input_ids'] = tokenized_prompt['input_ids']
    example['attention_mask'] = tokenized_prompt['attention_mask']
    example['labels'] = tokenized_summary['input_ids']

    return example

tokenizer.pad_token = tokenizer.eos_token
tokenized_datasets = dataset.map(tokenize_inputs, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'dialogue', 'summary'])
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

In [5]:
print(tokenized_datasets['train'].shape)
print(tokenized_datasets['validation'].shape)
print(tokenized_datasets['test'].shape)

(148, 6)
(9, 6)
(9, 6)


### Create PEFT Model using LoRA

In [6]:
from peft import LoraConfig, get_peft_model, TaskType


lora_config = LoraConfig(
    r=32, # 8, 16, 32
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [7]:
peft_model = get_peft_model(model, peft_config=lora_config)

In [8]:
from huggingface_hub import notebook_login

notebook_login()

### Train PEFT Model

In [12]:
from transformers import TrainingArguments, Trainer

peft_training_args = TrainingArguments(
    output_dir="./bart-cnn-samsum-peft",  # local directory
    hub_model_id="ingeniumacademy/bart-cnn-samsum-peft",  # identifier on the Hub
    learning_rate=1e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    auto_find_batch_size=True,
    eval_strategy='epoch',
    logging_steps=10
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
peft_model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 411,009,024 || trainable%: 1.1481


In [14]:
peft_trainer.train()

### Save PEFT Adapter

In [ ]:
peft_trainer.push_to_hub()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

events.out.tfevents.1717538832.cc5be8c71c42.1105.0:   0%|          | 0.00/9.43k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ingeniumacademy/bart-cnn-samsum-peft/commit/a6a46c94ec2f64c90bdce0d5cfa990806d0d3a5a', commit_message='End of training', commit_description='', oid='a6a46c94ec2f64c90bdce0d5cfa990806d0d3a5a', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
def generate_summary(input, llm):
  input_prompt = f"""
                  Summarize the following conversation.

                  {sample}

                  Summary:
                  """

  input_ids = tokenizer(sample, return_tensors='pt')
  tokenized_output = llm.generate(input_ids=input_ids['input_ids'], min_length=30, max_length=200)
  output = tokenizer.decode(tokenized_output[0], skip_special_tokens=True)

  return output

### Reload & Test

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("ingeniumacademy/bart-cnn-samsum-finetuned")

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("ingeniumacademy/bart-cnn-samsum-finetuned")

loaded_peft_model = PeftModel.from_pretrained(
    peft_model_base,
    "ingeniumacademy/bart-cnn-samsum-peft",
    is_trainable=False
)

adapter_config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

In [ ]:
sample = dataset['test'][0]['dialogue']
label = dataset['test'][0]['summary']

output = generate_summary(sample, llm=loaded_peft_model)

print("Sample")
print(sample)
print("-------------------")
print("Summary:")
print(output)
print("Ground Truth Summary:")
print(label)

Sample
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
-------------------
Summary:
Amanda can't find Betty's number. She'll ask Larry. He called her last time they were at the park together. He's very nice.
Ground Truth Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.
